In [1]:
import pandas as pd
import numpy as np
from tqdm import notebook
import os 

os.getcwd()

'/Users/yangwoolee/git_repo/seoul_bike_dash/moon_light'

> ### sub 기준 가까운 bike 정보 넣기

In [2]:
bike_station = pd.read_csv(
            "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
        ).reset_index()

sub_station = pd.read_csv(
            "assets/sub_station.csv", encoding="CP949", index_col=0
        ).reset_index()
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        np.sin(dlat / 2.0) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    )

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m

In [3]:
bikes_500 = []
for index, sub in notebook.tqdm(sub_station.iterrows()) :
    # print(sub)
    lon1 = sub['sub_경도']
    lat1 = sub['sub_위도']
    lon2 = bike_station['longtitude']
    lat2 = bike_station['latitude']
    sub_info = sub['역사_ID']
    a = haversine_np(lon1,lat1,lon2,lat2)
    b = bike_station[a<=500]['st_id']
    bikes_500.append(b.to_list())


# sub_station.drop(columns='1000m')
sub_station['500m'] = bikes_500
sub_station.columns = ['sub_id', 'sub_name', 'line', 'longtitude', 'latitude', 'bike_id']
sub_station['sub_name'] = sub_station['sub_name'] +'역'

sub_station


0it [00:00, ?it/s]

,sub_id,sub_name,line,sub_longtitude,sub_latitude,bike_id
0,9996,미사역,5호선,127.193877,37.560927,[]
1,9995,강일역,5호선,127.175930,37.557490,"[1016, 1057, 3676, 3689, 3690, 3694]"
2,4929,김포공항역,김포골드라인,126.801868,37.562360,[5072]
3,4928,고촌역,김포골드라인,126.770345,37.601243,[]
4,4927,풍무역,김포골드라인,126.732387,37.612488,[]
...,...,...,...,...,...,...
733,154,종로5가역,1호선,127.001849,37.570926,"[338, 339, 345, 387, 397, 452, 453, 455, 457, ..."
734,153,종로3가역,1호선,126.991847,37.570406,"[328, 329, 334, 335, 338, 397, 398, 454, 472, ..."
735,152,종각역,1호선,126.982923,37.570161,"[305, 316, 318, 320, 328, 329, 368, 385, 433, ..."
736,151,시청역,1호선,126.977088,37.565715,"[309, 310, 368, 370, 391, 433, 458, 466, 468, ..."


In [4]:
sub_station.to_csv('sub_and_bike_info.csv', encoding='CP949')

> ### search_box_csv 만들기

In [2]:
sub_info = pd.read_csv('assets/sub_and_bike_info.csv',encoding='CP949',index_col=0)
bus_info = pd.read_csv(
            "assets/near_bus_500m.csv", encoding="CP949", index_col=0
        )
bike_info = pd.read_csv(
            "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
        )
bike_info.reset_index(inplace=True)

bike_id = bike_info.index # 0~2000번대 자전거
sub_id = sub_info.index+3000 # 3000번대 지하철
bus_id = bus_info.index+4000 # 4~5000번대 버스

sub_name = sub_info['line'] + ' ' + sub_info['sub_name']
bus_name = bus_info['bus'] + ' ' + bus_info['name']
a=pd.concat([pd.Series(bike_id),bike_info['st_name']],axis=1)
a.columns = ['id','name']
b=pd.concat([pd.Series(sub_id),sub_name],axis=1)
b.columns = ['id','name']
c=pd.concat([pd.Series(bus_id),bus_name],axis=1)
c.columns = ['id','name']
d = pd.concat([a,b,c])

# d = 검색창 정보 


In [3]:
d.to_csv('경로',encoding='CP949')

,id,name
0,0,망원역 1번출구 앞
1,1,망원역 2번출구 앞
2,2,합정역 1번출구 앞
3,3,합정역 5번출구 앞
4,4,합정역 7번출구 앞
...,...,...
1847,5847,N75 세명컴퓨터고등학교
1848,5848,N75 박석고개.신도고등학교
1849,5849,N75 구파발역2번출구
1850,5850,N75 은평뉴타운10단지
